In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
# import duckdb as dd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
from pydicom import dcmread
import warnings
from sklearn.preprocessing import LabelEncoder
import pickle
import gc
import ctypes
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

2024-08-10 09:24:46.180128: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-10 09:24:46.180246: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-10 09:24:46.336120: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
model = keras.models.\
load_model("/kaggle/input/keras_base_lnfn_l1_l2/keras/default/1/keras_base_left_neural_foraminal_narrowing_l1_l2.h5")

In [3]:
submission = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/sample_submission.csv')
submission['row_id'] = 'samples'
#submission

In [4]:
Test = True
config = {}

if Test:
    config['root_file_path'] = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'
    config['start'] = 10
    config['end'] = 110
    studies = os.listdir(config['root_file_path'])[config['start']:config['end']]
    test_dict = {}
    image_files = []

    for study in studies:
        for dirname, _, filenames in os.walk(config['root_file_path']+'/'+study):
            for filename in filenames:
                test_dict[os.path.join(dirname, filename).split('/')[-3]] = image_files
                image_files.append(os.path.join(dirname, filename))
else:
    config['root_file_path'] = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/'
    test_dict = {}
    image_files = []

    for dirname, _, filenames in os.walk(config['root_file_path']):
        for filename in filenames:
            test_dict[os.path.join(dirname, filename).split('/')[-3]] = image_files
            image_files.append(os.path.join(dirname, filename))

In [6]:
for key,value in test_dict.items():
    print("there are {1} files under {0}".format(key,len(value)))

there are 7586 files under 2532413137
there are 7586 files under 4031357862
there are 7586 files under 3731783147
there are 7586 files under 4115208111
there are 7586 files under 3588716120
there are 7586 files under 2608235425
there are 7586 files under 1020394063
there are 7586 files under 3563559233
there are 7586 files under 2563313484
there are 7586 files under 82066307
there are 7586 files under 829439070
there are 7586 files under 2864325627
there are 7586 files under 108348787
there are 7586 files under 1153238977
there are 7586 files under 1719776527
there are 7586 files under 2697637622
there are 7586 files under 177339056
there are 7586 files under 3573227658
there are 7586 files under 2925530521
there are 7586 files under 106310815
there are 7586 files under 2493610993
there are 7586 files under 2121891805
there are 7586 files under 1661682358
there are 7586 files under 1768692511
there are 7586 files under 1908143210
there are 7586 files under 247968996
there are 7586 file

In [8]:
for st in tqdm(test_dict.keys()):
    print(st)

100%|██████████| 100/100 [00:00<00:00, 102300.10it/s]

2532413137
4031357862
3731783147
4115208111
3588716120
2608235425
1020394063
3563559233
2563313484
82066307
829439070
2864325627
108348787
1153238977
1719776527
2697637622
177339056
3573227658
2925530521
106310815
2493610993
2121891805
1661682358
1768692511
1908143210
247968996
2584853409
2851207875
1782095928
2387323642
2718854859
3367650254
3537214277
1165361924
390498354
3311528906
2155667219
2026420722
3151371929
2605844245
2406108213
2324297179
1722993663
3318343342
2247118504
3426183113
1292979992
2241283141
377474930
4283570761
3889130987
1722308532
2287443993
1094879459
1013589491
3976403280
2988180557
3237383375
2034823515
1057173941
4075603869
1864762516
1934246813
1750792644
2557856398
3192842688
2607713777
1505795551
332284668
339664659
3740224888
4161720216
1195410745
610806768
3824720894
52695609
3718047621
2799878304
2090173285
1606523187
3240929780
545140827
1670552101
113121178
3129894645
3293419987
2946997448
3587842489
2538919007
454297464
2379426952
1805845915
22829

In [12]:
for file in test_dict['2532413137']:
    print(len(file))

106
106
105
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
105
106
105
106
106
105
105
106
106
106
105
105
106
106
106
106
106
106
105
106
106
106
105
106
106
106
106
106
105
106
106
106
106
105
106
105
105
105
105
105
106
105
106
105
106
106
106
105
106
106
106
106
105
106
105
105
105
105
105
106
105
106
105
106
105
105
104
105
105
105
105
105
104
105
104
104
104
105
104
104
105
104
105
105
104
105
106
105
106
106
106
105
106
105
105
105
105
105
105
105
106
106
105
106
106
106
105
106
105
105
105
105
105
105
105
106
105
105
104
105
105
105
105
105
105
105
105
104
105
104
104
104
105
104
104
105
104
105
105
104
105
106
106
105
106
106
106
106
106
106
106
106
106
106
106
106
106
105
106
105
106
105
105
106
106
105
105
106
106
106
105
106
106
105
106
106
106
106
105
106
106
106
106
105
106
105
105
105
105
105
105
106
105
106
105
104
105
105
105
105
104
105
104
104
104
104
104
104
105
104
105
106
106
105
106
106
106
106
106
106
106
106
106
106
106
106


In [32]:
def get_feature_array(img_file_list):
    feature_resized = []
    for file in img_file_list:
        pbar.update(1)
        dicom_ds = dcmread(file)
        img_array = dicom_ds.pixel_array
        feature_resized.append(np.resize(np.mean(img_array.T, axis=0),(128,)))
    return feature_resized

vfunc = np.vectorize(get_feature_array, otypes=[list], signature='(n)->()')

In [ ]:
rows = {}
global pbar 
pbar = 0
for st in test_dict.keys():
    print(st)
    with tqdm(total=len(test_dict[st])) as pbar:
        final_feature_list = vfunc(test_dict[st])
    final = np.array(final_feature_list)
    y_proba = model.predict(final)
    rows[st] = np.mean(y_proba, axis=0)

2532413137


 51%|█████     | 3833/7586 [00:43<00:27, 137.38it/s]

In [5]:
rows = {}
# weight_dict = {'normal_mild':1, 'moderate':2, 'severe':4}
conditions = ['spinal_canal_stenosis', 'neural_foraminal_narrowing', 'subarticular_stenosis']
sides = ['left', 'right']
vertebrae_levels = ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']
severity_levels = ['normal_mild', 'moderate', 'severe']

for c in conditions:
    for v in vertebrae_levels:
        if c != 'spinal_canal_stenosis':
            for s in sides:
                for st in test_dict.keys():
                    if s+'_'+c+'_'+v == 'left_neural_foraminal_narrowing_l1_l2':
                        #print("going for model ", st+'_'+s+'_'+c+'_'+v)
                        #features = []
                        feature_resized = []
                        for file in tqdm(test_dict[st]):
                            dicom_ds = dcmread(file)
                            img_array = dicom_ds.pixel_array
                            feature_resized.append(np.resize(np.mean(img_array.T, axis=0),(128,)))
                        final = np.array(feature_resized)
                        y_proba = model.predict(final)
                        rows[st+'_'+s+'_'+c+'_'+v] = np.mean(y_proba, axis=0)
                    else:
                        #print(st+'_'+s+'_'+c+'_'+v)
                        rows[st+'_'+s+'_'+c+'_'+v] = np.array([0.333333 * 2, 0.333333 * 2, 0.333333 * 2])
        else:
            for st in test_dict.keys():
                #print(st+'_'+c+'_'+v)
                rows[st+'_'+c+'_'+v] = np.array([0.333333 * 2, 0.333333 * 2, 0.333333 * 2])

100%|██████████| 7586/7586 [03:04<00:00, 41.07it/s]


238/238 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


100%|██████████| 7586/7586 [02:04<00:00, 60.82it/s] 


238/238 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


100%|██████████| 7586/7586 [02:06<00:00, 60.07it/s] 


...

238/238 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


100%|██████████| 7586/7586 [02:08<00:00, 58.88it/s] 


238/238 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


 54%|█████▍    | 4089/7586 [01:34<01:20, 43.32it/s] 


KeyboardInterrupt: 

In [ ]:
for row_id, feature in tqdm(rows.items()):
    feature_set_reshaped = feature.reshape(1, -1)
    predictions = np.ascontiguousarray(feature_set_reshaped/2)
    df = pd.DataFrame(predictions, columns=severity_levels)
    df.insert(loc=0, column='row_id', value=row_id)
    submission = pd.concat([submission,df]).reset_index(drop=True)
    
i = submission[(submission.row_id == 'samples')].index
submission = submission.drop(i).reset_index(drop=True)

In [ ]:
submission.shape